In [1]:
#Setup Cell

import gdspy
import numpy

#creates a cell for the TWPA to live in. Required for the gdsII format.
lib = gdspy.GdsLibrary()
cell = lib.new_cell('TWPA')

In [2]:
#This Cell is the Control Cell were you input all values you control for the creation of a TWPA

#padlength is the x-axis extent of the TWPA's ground plane, padwidth is the y-axis extent of the TWPA's ground plane. Or, if no ground plane is generated, padlength is the distance between 
#wirebond pad ends.
padlength = 10000
padwidth = padlength

#wirbondpadlength is the x-axis extent of the wire-bond pads at the edge of the TWPA
#wirbondpadwidth is the y-axis extent of the wire-bond pads at the edge of the TWPA.
#wirbondpadgap is the spacing between the horizontal edges of the wire-bond pads and the ground plane.
#wirebondtaperlength is the length of the tapered line between the wire-bond pads and the lead lines.
#wirebondcentertoedge is the distance from the top (bottom) edge to the center of the input (output) line
wirebondpadlength = 500
wirebondpadwidth = 500
wirebondpadgap = 197
wirebondpadtaperlength = 500
wirebondcentertoedge = 1000

#inputoutputturnradius is the radius of the three-quarter circle that connects the lead lines to the center spiral.
inputoutputturnradius = 300

#radius is the outer radius of the center spiral
#radiusradio is the ratio between the inner and outer radius of the center spiral (i.e inner radius = radiusratio*radius).
#turns is the number of turns the center spiral has
radius = 3750
radiusratio = 0.1
turns = 10

#segments is the number of seperate pieces in a unit cell of the desired photonic crystal.
#For example: a repeating pattern of a short wide segment and a long narrow segment would have segments = 2
segments = 12

#lengthfunction is what defines the length of each of the segments segments and their order counting up from zero. 
#When fed an integer that corresponds to a segment, the functon returns the length of that segment.
def lengthfunction(n):
    if (n%4 == 0):
        y = 178
    if ((n%4 == 1) or (n%4 == 3)):
        y = 1
    if (n == 2 or n == 6):
        y = 30
    if (n == 10):
        y = 60
    return y

#widthfunction is what defines the width of each of the segments segments and their order counting up from zero. The order of the widths should correspond to the order set in lengthfunction.
#When fed an integer that corresponds to a segment and the fractional position along the spiral (i.e. x=0 at the start of spiral, x=1 at the end of a single spiral where a spiral starts at the 
#end of the wirebondpad-taper and ends at the start of the center straight-and-turns area) the function returns the width of that segment.
def widthfunction(n, x):
    if (n%4 == 2):
        y = 2*(25*numpy.cos(numpy.pi*x*500) + 30)
    else:
        y = 25*numpy.cos(numpy.pi*x*500) + 30
    return y

#gapfunction is what defines the gap of each of the segments segments and their order counting up from zero. 
#Order of the gaps should correspond to the order set in lengthfunction and widthfunction.
#When fed an integer that corresponds to a segment and the fractional position along the spiral (i.e. x=0 at the start of spiral, x=1 at the end of the spiral where a spiral starts at the 
#end of the wirebondpad-taper and ends at the start of the center straight-and-turns area) the functon returns the gap between the center line and the ground plane of that segment.
def gapfunction(n, x): 
    if (n%4 == 2):
        y = 2*(25*numpy.cos(numpy.pi*x*500) + 30)
    if ((n%4 == 1) or (n%4 == 3)):
        y = 1.5*(25*numpy.cos(numpy.pi*x*500) + 30)
    else:
        y = 25*numpy.cos(numpy.pi*x*500) + 30
    return y

#spirallaterratio is the proportion of the length of a spiral that tapers from the initially defined (x = 0) values to the final (x = 1) defined values
spiraltaperratio = 1 #20000/(numpy.pi*(turns+0.5)*0.5*((1 + radiusratio)*radius) + padlength/2 - wirebondpadlength - wirebondpadtaperlength + radius - inputoutputturnradius + padlength/2 - wirebondcentertoedge - inputoutputturnradius)

#tol is how closely gdspy approximates a curve with polygons. Lower is more accurate but takes much longer to generate
#numeval is the starting number of evaluations that each straight section of the twpa line will expereince, scaled by tolerance in some way
tol = 0.00001 #0.00001 for relatively quick TWPA <~1m add a 0 for <~2m add another for <~5m
numeval = int(padlength/2 - wirebondpadlength - wirebondpadtaperlength + radius - inputoutputturnradius)

#The number of vertical slices that the ground plane will be split into, affects the time for the TWPA to load, more is better to a point
slices = 50

In [3]:
#This Cell holds misc functions and values calculated from the values in the Control Cell

#Calculates the length of a single unit of photonic crystal and stores it as unitlength
unitlength = 0
for n in range(segments):
    unitlength += lengthfunction(n)

#Given the length x, returns the number of the segement within a unit. x must be 0 <= x <= unitlength
def locationinunit(x):
    pos = 0
    location = -1
    for n in range(segments):
        if (pos <= x) and (x < pos + lengthfunction(n)):
            location = n
        pos += lengthfunction(n)
    return location

#Returns the end length of the nth segment within a unit
def endlengthofsegment(n):
    endx = 0
    for c in range(n):
        endx += lengthfunction(c)
    return endx

#inputoutputhorizontallength is the length of the horizontal lines leading to and from the wirebondad-taper to the central sprial
#inputoutputverticallength is the length of the vertical lines leading to and from the wirebondad-taper to the central sprial
inputoutputhorizontallinelength = padlength/2 - wirebondpadlength - wirebondpadtaperlength + radius - inputoutputturnradius
inputoutputverticallinelength = padlength/2 - wirebondcentertoedge - inputoutputturnradius

#radius, angle, and derivitives to calculate the next four values
r1 = radius - (1 - radiusratio)*radius
theta1 = (turns+0.5) * numpy.pi
dx_du1 = (1-radiusratio)*radius*numpy.cos(theta1) + r1*numpy.sin(theta1)*(turns+0.5)*numpy.pi
dy_du1 = -(1-radiusratio)*radius*numpy.sin(theta1) + r1*numpy.cos(theta1)*(turns+0.5)*numpy.pi

#values calculated to make sure the center S portion of the TWPA connects and aligns correctly.
#dy_dx1 is the derivitive of the center of the spirals' ends.
#spiralendangle is the angle that the ends of the spiral make relative to the veritcal plane
#centerradius is the radius that the center turns need to have in order to conenct with the center segments in order for the segments to connect with the center ends of the spirals
#centerstarightength is the length that the center segments need to have in order to conenct with the center turns and the center ends of the spirals
dy_dx1 = dy_du1/dx_du1
spiralendangle = numpy.arctan(dy_dx1)
centerturnradius = (radiusratio*radius)/(1 + numpy.cos(spiralendangle) + numpy.sin(spiralendangle)*dy_dx1)

#singlespirallength is the length of a single spiral (ie. the length along the spiral from the end of the outer edge of the spiral to the inner edge), including relevant inputoutput lines and turn
singlespirallength = numpy.pi*(turns+0.5)*0.5*((1 + radiusratio)*radius) + inputoutputhorizontallinelength + inputoutputturnradius*numpy.pi/2 + inputoutputverticallinelength - centerturnradius*dy_dx1 + centerturnradius*(numpy.pi + spiralendangle)

#unitsperspiral is the number of unit cells of photonic crystal constained within a single spiral, including the relevant inputoutput lines and turn
unitsperspiral = (singlespirallength)/unitlength

#unitremainder is the fractional part of unitsperspiral
#unitremainder is the length of the parital unit contained within a single spiral, including the relevant inputoutput lines and turn
unitremainder = unitsperspiral - numpy.trunc(unitsperspiral)
unitremainderlength = unitremainder * unitlength

#totallength is the center length of the TWPA's center line from the end of one spiral to the end of the other
#totals full units is the number of full photonic crystal unit cells withing the center spiral portion of the TWPA
totallength = 2*(singlespirallength + centerturnradius*dy_dx1 + 2*centerturnradius*numpy.pi)
totalfullunits = numpy.trunc(totallength/unitlength)

#thinwirelength is the length of the center TWPA line between the ends of the tapers.
#thinfullunits is the number of full photonic crystal units that can fit on the line between tapers.
thinwirelength = 2*((1 - spiraltaperratio)*singlespirallength + centerturnradius*dy_dx1 + 2*centerturnradius*numpy.pi)
thinfullunits = numpy.trunc(thinwirelength/unitlength)

#spacing is the space between centers of the turns of the center spiral
spacing = ((1 - radiusratio)*radius)/(turns + 0.5)

#a function that takes two arguments and always returns 0. Added to make the gapfunc 0 when making the thin center line and not the ground plane with the thicker center line cutout.
def zerofunction(a, b):
    return 0

In [4]:
#This Cell holds all the code that greates the TWPA line and Ground plane in a function called centerline

#creates the center line of a TWPA that has width widthfunc + 2*gapfunc. If makegroundplane=1 centerline will subtract the centerline from a rectangle to serve as a ground plane and add it to the cell, 
#if makegroundplane=0 centerline will add the center line to the cell.
def centerline(widthfunc, gapfunc, makegroundplane): 
    
    #creates the TWPAinputline path object with an initial width of wirebondpadwidth + 2*wirebondpadgap*makegroundplane and a start position of (0, padlength/2 - wirebondcentertoedge) i.e. the top left corner
    TWPAinputline = gdspy.Path(wirebondpadwidth + 2*wirebondpadgap*makegroundplane, (0, padlength/2 - wirebondcentertoedge))
    
    #adds a straight segment of length wirebondpathlangth in the +x direction to the end of the path. Since a new width was not specified the segment will use the path's initial width
    TWPAinputline.segment(wirebondpadlength, '+x')

    #adds a straight segment of length wirebondtaperlength to the end of the TWPAinputline path. 
    #Since a new direction was not specified the segment will be oriented in the same direction as the end of the previous part of the path.
    #Since a final_width was defined, the segment will linearly taper from the final_width of the previous part of the path to the defined final_width of widthfunc(0, 0) + 2*gapfunc(0, 0)
    TWPAinputline.segment(wirebondpadtaperlength, final_width = widthfunc(0, 0) + 2*gapfunc(0, 0))

    #widfuncinputoutputhorizontaline is the function which defines horizontal input line's width at a specific value of u. lengthalongspiral is the length of spiral created at a given value of u,
    #and lengthalong unit is how far along a unit cell a given value of u is. If the fraction lengthalongspiral/singlespirallength is less than spiraltaperratio the x value of 
    #(lengthalongspiral/singlespirallength)/spiraltaperratio) is given to widthfunction to taper as defined, if not, the x value of 1 (end of taper) is fed instead. locationinunit(lengthalongunit) gives
    #the n value of the correct segment for a given value of u. This function the returns the correct spiral width for a given value of u.
    def widfuncinputhorizontalline(u):
        lengthalongspiral = u*inputoutputhorizontallinelength
        lengthalongunit = (lengthalongspiral)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #This function defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def inputhorizontalline(u):
        x = u*(inputoutputhorizontallinelength)
        y = 0
        return(x, y)

    #Uses the two above functions to create a horizontal line.
    TWPAinputline.parametric(inputhorizontalline, tolerance = tol, max_points = 8190, final_width= widfuncinputhorizontalline, relative = True, number_of_evaluations = numeval)

    #widfuncinputoutputturn is the function which defines input turn's width at a specific value of u. lengthalongspiral is the length of spiral created at a given value of u,
    #and lengthalong unit is how far along a unit cell a given value of u is. If the fraction lengthalongspiral/singlespirallength is less than spiraltaperratio the x value of 
    #(lengthalongspiral/singlespirallength)/spiraltaperratio) is given to widthfunction to taper as defined, if not, the x value of 1 (end of taper) is fed instead. locationinunit(lengthalongunit) gives
    #the n value of the correct segment for a given value of u. This function the returns the correct spiral width for a given value of u.
    def widfuncinputturn(u):
        lengthalongspiral = u*(inputoutputturnradius*numpy.pi/2) + inputoutputhorizontallinelength
        lengthalongunit = (lengthalongspiral)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #Defines a right turn
    def inputturn(u):
        r = inputoutputturnradius
        theta = u*(numpy.pi/2)
        y = r*(numpy.cos(theta) - 1)
        x = r*numpy.sin(theta)
        return(x, y)
    
    #Defines the derivitive of a right turn
    def dinputturn_du(u):
        r = inputoutputturnradius
        theta = u*(numpy.pi/2)
        dy = -r*numpy.sin(theta)
        dx = r*numpy.cos(theta)
        return(dx, dy)
    
    #Uses the above three functions to create a right turn
    TWPAinputline.parametric(inputturn, dinputturn_du, max_points= 8190, final_width= widfuncinputturn, relative= True)

    #widfuncinputoutputverticalline is the function which defines vertical input line's width at a specific value of u. lengthalongspiral is the length of spiral created at a given value of u,
    #and lengthalong unit is how far along a unit cell a given value of u is. If the fraction lengthalongspiral/singlespirallength is less than spiraltaperratio the x value of 
    #(lengthalongspiral/singlespirallength)/spiraltaperratio) is given to widthfunction to taper as defined, if not, the x value of 1 (end of taper) is fed instead. locationinunit(lengthalongunit) gives
    #the n value of the correct segment for a given value of u. This function the returns the correct spiral width for a given value of u.
    def widfuncinputverticalline(u):
        lengthalongspiral = u*inputoutputverticallinelength + inputoutputturnradius*numpy.pi/2 + inputoutputhorizontallinelength
        lengthalongunit = (lengthalongspiral)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #This function defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def inputvertialline(u):
        x = 0
        y = -u*(inputoutputverticallinelength)
        return(x, y)

    #Uses the above two functions to create a vertical line
    TWPAinputline.parametric(inputvertialline, tolerance = tol, max_points = 8190, final_width= widfuncinputverticalline, relative = True, number_of_evaluations = numeval)
    
    #creates the TWPAspiralline path object with an initial width of widthfunc(0, 0) + 2*gapfunc(0, 0) and a start position of (-radius, 0) which places the center of the spiral at (0, 0)
    TWPAspiralline = gdspy.Path(widthfunc(0, 0) + 2*gapfunc(0, 0), (-radius, 0))
    
    #spiral2 is the function that defines the spiral's shape. It is a parametric function of u, a variable which varies from 0 to 1. 
    #First the polar equation is defined, then it is translated into cartesian coordinates which are returned by the function
    def spiral1(u):
        r = radius - (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi
        x = -r * numpy.cos(theta) + radius
        y = r * numpy.sin(theta)
        return (x, y)

    #dspiral1_du is the function which defines the spiral's derivitive. The path's Parametric method uses the derivitive to calculated the direction of the vector of the parametric portion of the path.
    #to simplify the start of the spiral a condition has been added which forces dspiral1_du(0) to be (0, 1) so that the start face of the spiral has a normal vector point in the +y direction, 
    #which results in the start face being horizontal, therefore fully connecting to the input portion of the design. If a derivitive function is not defined, the parametric method will calculate the
    #derivitive numerically from the defined function.
    def dspiral1_du(u):
        r = radius - (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi
        if u == 0:
            dx = 0
            dy = 1
        else:
            dx = (1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi*u) + r*numpy.sin((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi
            dy = -(1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi*u) + r*numpy.cos((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi
        return (dx, dy)

    #widfuncspiral1 is the function which defines the spiral's width at a specific value of u. lengthalongspiral is the length of spiral created at a given value of u,
    #and lengthalong unit is how far along a unit cell a given value of u is. If the fraction lengthalongspiral/singlespirallength is less than spiraltaperratio the x value of 
    #(lengthalongspiral/singlespirallength)/spiraltaperratio) is given to widthfunction to taper as defined, if not, the x value of 1 (end of taper) is fed instead. locationinunit(lengthalongunit) gives
    #the n value of the correct segment for a given value of u. This function the returns the correct spiral width for a given value of u.
    def widfuncspiral1(u):
        lengthalongspiral = numpy.pi*(turns+0.5)*0.5*radius*u*(2-(1 - radiusratio)*u) + inputoutputhorizontallinelength + inputoutputverticallinelength + inputoutputturnradius*numpy.pi/2
        lengthalongunit = (lengthalongspiral)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #uses the three previously defined functions to add the desired spiral to the end of the TWPAspiralline path. Each of the functions is fed a value of u, which varies from 0 to 1, to sweep out the shape.
    #Since gdsII files only contain polygons and a spiral is a curve which much be approximated by polygons, tolerance is specified as well as the maximum number of vertices a polygon is allowed
    #to have. relaive = True allows the paametric shape to be moved so that it starts at the previous end of the path, but only if u=0 results in each of the defined functions returning (0, 0).
    TWPAspiralline.parametric(spiral1, dspiral1_du, tolerance = tol, max_points = 8190, final_width = widfuncspiral1, relative = True)
    
    #This function defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def segment1(u):
        if  (turns%2) == 1:
            r = u*centerturnradius*dy_dx1
        else:
            r = -u*centerturnradius*dy_dx1
        x = r*numpy.cos(spiralendangle)
        y = r*numpy.sin(spiralendangle)
        return (x,y)

    #Defines the width of the parametric segment for varous values of u. 
    def widfuncsegment1(u):
        lengthalongspiral = numpy.pi*(turns+0.5)*0.5*radius*(2-(1 - radiusratio)) + inputoutputhorizontallinelength + inputoutputverticallinelength + inputoutputturnradius*numpy.pi/2 - centerturnradius*dy_dx1*u
        lengthalongunit = (lengthalongspiral + unitremainderlength)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #adds a parametric line defined above to the end of the TWPAspiralline path. This segment is parametric due to normal path.segment objects only being able to linearly taper their widths instead of 
    #continuously varying them.
    TWPAspiralline.parametric(segment1, final_width = widfuncsegment1, max_points = 8190, tolerance = tol, relative = True, number_of_evaluations = numeval)
    
    #defines a parametric turn with a start angle of spiralendangle + numpy.pi*(turns%2) and an end angle of (numpy.pi + spiralendangle) + spiralendangle + numpy.pi*(turns%2) and constant radius
    #of centerturnradius. The start angle was chosen to keep the turn and previous segment flush due to the archimedian spiral's end face not being vertially aligned.
    def turn1(u):
        theta = -u * (numpy.pi + spiralendangle) + spiralendangle + numpy.pi*(turns%2)
        r = centerturnradius
        x = -r * numpy.sin(theta) + r * numpy.sin(spiralendangle + numpy.pi*(turns%2))
        y = r * numpy.cos(theta)  - r * numpy.cos(spiralendangle + numpy.pi*(turns%2))
        return (x, y)
    
    #defines the first turn's derivitive
    def dturn1_du(u):
        theta = -u * (numpy.pi + spiralendangle) + spiralendangle + numpy.pi*(turns%2)
        r = centerturnradius
        if u > 0:
            dx = -r*(theta/u)*numpy.cos(theta)
            dy = -r*(theta/u)*numpy.sin(theta)
        else:
            dx = (1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi) + radius*radiusratio*numpy.sin((turns+0.5)*numpy.pi)*(turns+0.5)*numpy.pi
            dy = -(1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi) + radius*radiusratio*numpy.cos((turns+0.5)*numpy.pi)*(turns+0.5)*numpy.pi
        return (dx, dy)
    
    #Defines the width of the parametric turn for varous values of u.
    def widfuncturn1(u):
        lengthalongspiral = numpy.pi*(turns+0.5)*0.5*radius*(2-(1 - radiusratio)) + inputoutputhorizontallinelength + inputoutputverticallinelength + inputoutputturnradius*numpy.pi/2 - centerturnradius*dy_dx1 + centerturnradius*(numpy.pi + spiralendangle)*u
        lengthalongunit = (lengthalongspiral + unitremainderlength - centerturnradius*dy_dx1)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #Adds the parametric turn defined above to the end of the TWPAspiralline path. This turn is parametric due to normal path.turn objects only being able to linearly taper their widths instead of 
    #continuously varying them.
    TWPAspiralline.parametric(turn1, dturn1_du, tolerance = tol, max_points = 8190, final_width = widfuncturn1, relative = True)
    
    #defines a parametric turn with a start angle of numpy.pi*(turns%2) and an end angle of (numpy.pi + spiralendangle) + numpy.pi*(turns%2) and constant radius
    #of centerturnradius. The end angle was chosen to keep the next segment and next spiral flush due to the archimedian spiral's end face not being vertially aligned.
    def turn2(u):
        r = centerturnradius
        theta = u * (numpy.pi + spiralendangle) + numpy.pi*(turns%2)
        x = -r * numpy.sin(theta) + r*numpy.sin(numpy.pi*(turns%2))
        y = r * numpy.cos(theta) - r*numpy.cos(numpy.pi*(turns%2))
        return (x, y)
    
    #defines the second turn's derivitive
    def dturn2_du(u):
        theta = u * (numpy.pi + spiralendangle)
        r = centerturnradius
        if u < 1:
            dx = -r*(numpy.pi + spiralendangle)*numpy.cos(theta)
            dy = -r*(numpy.pi + spiralendangle)*numpy.sin(theta)
        else:
            dx = radiusratio*radius*(turns+0.5)*numpy.pi
            dy = -(1 - radiusratio)*radius
        return (dx, dy)
    
    #Defines the width of the parametric turn for varous values of u.
    def widfuncturn2(u):
        lengthalongspiral = centerturnradius*(numpy.pi + spiralendangle)*u
        lengthalongunit = (lengthalongspiral + unitremainderlength - centerturnradius*dy_dx1 + centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #Adds a parametric turn defined above to the end of the TWPAspiralline path.
    TWPAspiralline.parametric(turn2, dturn2_du, tolerance = tol, max_points = 8190, final_width = widfuncturn2)
    
    #Defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def segment2(u):
        if  (turns%2) == 1:
            r = u*centerturnradius*dy_dx1
        else:
            r = -u*centerturnradius*dy_dx1
        x = r*numpy.cos(spiralendangle)
        y = r*numpy.sin(spiralendangle)

        return (x,y)

    #Defines the width of the parametric segment for varous values of u.
    def widfuncsegment2(u):
        lengthalongspiral = centerturnradius*(numpy.pi + spiralendangle) - centerturnradius*dy_dx1*u
        lengthalongunit = (lengthalongspiral + unitremainderlength - centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #adds the parametric line defined above to the end of the TWPAspiralline path.
    TWPAspiralline.parametric(segment2, final_width = widfuncsegment2, tolerance = tol, max_points = 8190, relative = True, number_of_evaluations = numeval)
    
    #defines the second spiral's shape. This spiral starts on its inner radius and spirals outwads to its outer radius.
    def spiral2(u):
        r = radiusratio*radius + (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi + numpy.pi*(turns%2)
        x = r * numpy.sin(theta) - r*numpy.sin(numpy.pi*(turns%2))
        y = -r * numpy.cos(theta) - radiusratio*radius*(turns%2) + radiusratio*radius*((turns+1)%2)
        return (x, y)

    #defines the derivitive of the second spiral. There is a condition that the end of the spiral's normal vector is pointed in the +y direction.
    def dspiral2_du(u):
        r = radiusratio*radius + (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi
        if u == 1:
            dx = 0
            dy = 1
        else:
            if (turns%2) == 0:
                dx = ((1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi*u) + r*numpy.cos((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
                dy = (-(1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi*u) + r*numpy.sin((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
            else:
                dx = -((1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi*u) + r*numpy.cos((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
                dy = -(-(1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi*u) + r*numpy.sin((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
        return (dx, dy)

    #Defines the width as a function of u of the second spiral.
    def widfuncspiral2(u):
        lengthalongspiral = centerturnradius*(numpy.pi + spiralendangle) - centerturnradius*dy_dx1 + numpy.pi*(turns + 0.5)*0.5*radius*u*(2*radiusratio + (1 - radiusratio)*u)
        lengthalongunit = (lengthalongspiral + unitremainderlength - 2*centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #adds the second spiral defined above to the end of the TWPAspiralline path. 
    TWPAspiralline.parametric(spiral2, dspiral2_du, tolerance = tol, max_points = 8190, final_width = widfuncspiral2, relative = True)
    
    #Rotates the TWPAspiralline path pi-spiralcenterangle radians around its center (0, 0) then moves it to the center of the pad.
    TWPAspiralline.rotate(numpy.pi)
    TWPAspiralline.translate(padlength/2, 0)
    
    #Creates a new path called TWPAoutputline of width widthfunc(0, 0) + 2*gapfunc(0, 0) at (padwidth/2 - radius, 0)
    TWPAoutputline = gdspy.Path(widthfunc(0, 0) + 2*gapfunc(0, 0), (padwidth/2 - radius, 0))

    #Defines a vertical line
    def outputvertialline(u):
        x = 0
        y = -u*(padlength/2 - wirebondcentertoedge - inputoutputturnradius)
        return(x, y)
    
    #defines the width of the output vertical line as a function of u.
    def widfuncoutputverticalline(u):
        lengthalongspiral = singlespirallength - (1 - u)*inputoutputverticallinelength - (inputoutputturnradius*numpy.pi)/2 - inputoutputhorizontallinelength
        lengthalongunit =(lengthalongspiral + unitremainderlength - 2*centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #Adds the vertical line to the path
    TWPAoutputline.parametric(outputvertialline, tolerance = tol, max_points = 8190, final_width= widfuncoutputverticalline, relative = True, number_of_evaluations = numeval)

    #Defines a left turn
    def outputturn(u):
        r = inputoutputturnradius
        theta = u*(numpy.pi/2)
        x = -r*(numpy.cos(theta) - 1)
        y = -r*numpy.sin(theta)
        return(x, y)
    
    #Defines the derivitive of a left turn
    def doutputturn_du(u):
        r = inputoutputturnradius
        theta = u*(numpy.pi/2)
        dx = r*numpy.sin(theta)
        dy = -r*numpy.cos(theta)
        return(dx, dy)

    #Defines the width of a left turn as a function of u.
    def widfuncoutputturn(u):
        lengthalongspiral = singlespirallength - (1 - u)*(inputoutputturnradius*numpy.pi)/2 - inputoutputhorizontallinelength
        lengthalongunit =(lengthalongspiral + unitremainderlength - 2*centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #Adds a left turn to the path.
    TWPAoutputline.parametric(outputturn, doutputturn_du, tolerance = tol, max_points= 8190, final_width= widfuncoutputturn, relative= True)

    #Defines a horizontal ilne
    def outputhorizontalline(u):
        x = u*(inputoutputhorizontallinelength)
        y = 0
        return(x, y)
    
    #Defines the width of a horizontal line as a function of u.
    def widfuncoutputhorizontalline(u):
        lengthalongspiral = singlespirallength - (1 - u)*inputoutputhorizontallinelength
        lengthalongunit = (lengthalongspiral + unitremainderlength + 2*centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #Adds a horzinontal line to the path.
    TWPAoutputline.parametric(outputhorizontalline, tolerance = tol, max_points = 8190, final_width= widfuncoutputhorizontalline, relative = True, number_of_evaluations = numeval)

    #Adds a segment of length wirebondpadtaperlength that linearly tapers from the end width of the horizontal line to wirebondpadwith + 2*wirebondpadgap*makegroundplane
    TWPAoutputline.segment(wirebondpadtaperlength, final_width = wirebondpadwidth + 2*wirebondpadgap*makegroundplane)

    #Adds a segment of length wirebondpadlength and width wirebondpadwidth + 2*wirebondpadgap*makegroundplane
    TWPAoutputline.segment(wirebondpadlength, final_width= wirebondpadwidth + 2*wirebondpadgap*makegroundplane)

    #decide whether to add a rectange and subtract the TWPAinputline, TWPAspiralline and TWPAoutputline paths from it, or to add the TWPAinputline, TWPAspiralline and TWPAoutputline paths to the cell.
    if makegroundplane:
        
        groundrectangle = gdspy.Rectangle((0, padwidth/2), (padlength, -padwidth/2))
        
        positions = []
        
        for i in range(1, slices):
            positions.append((padlength/slices)*i)
        
        groundslice = gdspy.slice(groundrectangle, positions, 0, precision = tol)
        
        for i in range(slices):
            groundslice[i] = gdspy.boolean(groundslice[i], TWPAinputline, 'not', precision = tol, max_points = 8190)
        
            groundslice[i] = gdspy.boolean(groundslice[i], TWPAspiralline, 'not', precision = tol, max_points = 8190)

            groundslice[i] = gdspy.boolean(groundslice[i], TWPAoutputline, 'not', precision = tol, max_points = 8190)

            cell.add(groundslice[i])

    else:
        cell.add(TWPAinputline)
        
        cell.add(TWPAspiralline)
        
        cell.add(TWPAoutputline)
    
    return

In [5]:
#This Cell holds all the code that greates the TWPA line and Ground plane in a function called gapline
def gapline(widthfunc, gapfunc): 
    
    #creates the TWPAinputline path object with an initial width of wirebondpadwidth + 2*wirebondpadgap*makegroundplane and a start position of (0, padlength/2 - wirebondcentertoedge) i.e. the top left corner
    TWPAinputlinewide = gdspy.Path(wirebondpadwidth + 2*wirebondpadgap, (0, padlength/2 - wirebondcentertoedge))
    
    #adds a straight segment of length wirebondpathlangth in the +x direction to the end of the path. Since a new width was not specified the segment will use the path's initial width
    TWPAinputlinewide.segment(wirebondpadlength, '+x')

    #adds a straight segment of length wirebondtaperlength to the end of the TWPAinputline path. 
    #Since a new direction was not specified the segment will be oriented in the same direction as the end of the previous part of the path.
    #Since a final_width was defined, the segment will linearly taper from the final_width of the previous part of the path to the defined final_width of widthfunc(0, 0) + 2*gapfunc(0, 0)
    TWPAinputlinewide.segment(wirebondpadtaperlength, final_width = widthfunc(0, 0) + 2*gapfunc(0, 0))

    #widfuncinputoutputhorizontaline is the function which defines horizontal input line's width at a specific value of u. lengthalongspiral is the length of spiral created at a given value of u,
    #and lengthalong unit is how far along a unit cell a given value of u is. If the fraction lengthalongspiral/singlespirallength is less than spiraltaperratio the x value of 
    #(lengthalongspiral/singlespirallength)/spiraltaperratio) is given to widthfunction to taper as defined, if not, the x value of 1 (end of taper) is fed instead. locationinunit(lengthalongunit) gives
    #the n value of the correct segment for a given value of u. This function the returns the correct spiral width for a given value of u.
    def widfuncinputhorizontalline(u):
        lengthalongspiral = u*inputoutputhorizontallinelength
        lengthalongunit = (lengthalongspiral)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #This function defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def inputhorizontalline(u):
        x = u*(inputoutputhorizontallinelength)
        y = 0
        return(x, y)

    #Uses the two above functions to create a horizontal line.
    TWPAinputlinewide.parametric(inputhorizontalline, tolerance = tol, max_points = 8190, final_width= widfuncinputhorizontalline, relative = True, number_of_evaluations = numeval)

    #widfuncinputoutputturn is the function which defines input turn's width at a specific value of u. lengthalongspiral is the length of spiral created at a given value of u,
    #and lengthalong unit is how far along a unit cell a given value of u is. If the fraction lengthalongspiral/singlespirallength is less than spiraltaperratio the x value of 
    #(lengthalongspiral/singlespirallength)/spiraltaperratio) is given to widthfunction to taper as defined, if not, the x value of 1 (end of taper) is fed instead. locationinunit(lengthalongunit) gives
    #the n value of the correct segment for a given value of u. This function the returns the correct spiral width for a given value of u.
    def widfuncinputturn(u):
        lengthalongspiral = u*(inputoutputturnradius*numpy.pi/2) + inputoutputhorizontallinelength
        lengthalongunit = (lengthalongspiral)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #Defines a right turn
    def inputturn(u):
        r = inputoutputturnradius
        theta = u*(numpy.pi/2)
        y = r*(numpy.cos(theta) - 1)
        x = r*numpy.sin(theta)
        return(x, y)
    
    #Defines the derivitive of a right turn
    def dinputturn_du(u):
        r = inputoutputturnradius
        theta = u*(numpy.pi/2)
        dy = -r*numpy.sin(theta)
        dx = r*numpy.cos(theta)
        return(dx, dy)
    
    #Uses the above three functions to create a right turn
    TWPAinputlinewide.parametric(inputturn, dinputturn_du, max_points= 8190, final_width= widfuncinputturn, relative= True)

    #widfuncinputoutputverticalline is the function which defines vertical input line's width at a specific value of u. lengthalongspiral is the length of spiral created at a given value of u,
    #and lengthalong unit is how far along a unit cell a given value of u is. If the fraction lengthalongspiral/singlespirallength is less than spiraltaperratio the x value of 
    #(lengthalongspiral/singlespirallength)/spiraltaperratio) is given to widthfunction to taper as defined, if not, the x value of 1 (end of taper) is fed instead. locationinunit(lengthalongunit) gives
    #the n value of the correct segment for a given value of u. This function the returns the correct spiral width for a given value of u.
    def widfuncinputverticalline(u):
        lengthalongspiral = u*inputoutputverticallinelength + inputoutputturnradius*numpy.pi/2 + inputoutputhorizontallinelength
        lengthalongunit = (lengthalongspiral)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #This function defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def inputvertialline(u):
        x = 0
        y = -u*(inputoutputverticallinelength)
        return(x, y)

    #Uses the above two functions to create a vertical line
    TWPAinputlinewide.parametric(inputvertialline, tolerance = tol, max_points = 8190, final_width= widfuncinputverticalline, relative = True, number_of_evaluations = numeval)
    
    #creates the TWPAspiralline path object with an initial width of widthfunc(0, 0) + 2*gapfunc(0, 0) and a start position of (-radius, 0) which places the center of the spiral at (0, 0)
    TWPAspirallinewide = gdspy.Path(widthfunc(0, 0) + 2*gapfunc(0, 0), (-radius, 0))
    
    #spiral2 is the function that defines the spiral's shape. It is a parametric function of u, a variable which varies from 0 to 1. 
    #First the polar equation is defined, then it is translated into cartesian coordinates which are returned by the function
    def spiral1(u):
        r = radius - (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi
        x = -r * numpy.cos(theta) + radius
        y = r * numpy.sin(theta)
        return (x, y)

    #dspiral1_du is the function which defines the spiral's derivitive. The path's Parametric method uses the derivitive to calculated the direction of the vector of the parametric portion of the path.
    #to simplify the start of the spiral a condition has been added which forces dspiral1_du(0) to be (0, 1) so that the start face of the spiral has a normal vector point in the +y direction, 
    #which results in the start face being horizontal, therefore fully connecting to the input portion of the design. If a derivitive function is not defined, the parametric method will calculate the
    #derivitive numerically from the defined function.
    def dspiral1_du(u):
        r = radius - (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi
        if u == 0:
            dx = 0
            dy = 1
        else:
            dx = (1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi*u) + r*numpy.sin((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi
            dy = -(1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi*u) + r*numpy.cos((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi
        return (dx, dy)

    #widfuncspiral1 is the function which defines the spiral's width at a specific value of u. lengthalongspiral is the length of spiral created at a given value of u,
    #and lengthalong unit is how far along a unit cell a given value of u is. If the fraction lengthalongspiral/singlespirallength is less than spiraltaperratio the x value of 
    #(lengthalongspiral/singlespirallength)/spiraltaperratio) is given to widthfunction to taper as defined, if not, the x value of 1 (end of taper) is fed instead. locationinunit(lengthalongunit) gives
    #the n value of the correct segment for a given value of u. This function the returns the correct spiral width for a given value of u.
    def widfuncspiral1(u):
        lengthalongspiral = numpy.pi*(turns+0.5)*0.5*radius*u*(2-(1 - radiusratio)*u) + inputoutputhorizontallinelength + inputoutputverticallinelength + inputoutputturnradius*numpy.pi/2
        lengthalongunit = (lengthalongspiral)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #uses the three previously defined functions to add the desired spiral to the end of the TWPAspiralline path. Each of the functions is fed a value of u, which varies from 0 to 1, to sweep out the shape.
    #Since gdsII files only contain polygons and a spiral is a curve which much be approximated by polygons, tolerance is specified as well as the maximum number of vertices a polygon is allowed
    #to have. relaive = True allows the paametric shape to be moved so that it starts at the previous end of the path, but only if u=0 results in each of the defined functions returning (0, 0).
    TWPAspirallinewide.parametric(spiral1, dspiral1_du, tolerance = tol, max_points = 8190, final_width = widfuncspiral1, relative = True)
    
    #This function defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def segment1(u):
        if  (turns%2) == 1:
            r = u*centerturnradius*dy_dx1
        else:
            r = -u*centerturnradius*dy_dx1
        x = r*numpy.cos(spiralendangle)
        y = r*numpy.sin(spiralendangle)
        return (x,y)

    #Defines the width of the parametric segment for varous values of u. 
    def widfuncsegment1(u):
        lengthalongspiral = numpy.pi*(turns+0.5)*0.5*radius*(2-(1 - radiusratio)) + inputoutputhorizontallinelength + inputoutputverticallinelength + inputoutputturnradius*numpy.pi/2 - centerturnradius*dy_dx1*u
        lengthalongunit = (lengthalongspiral + unitremainderlength)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #adds a parametric line defined above to the end of the TWPAspiralline path. This segment is parametric due to normal path.segment objects only being able to linearly taper their widths instead of 
    #continuously varying them.
    TWPAspirallinewide.parametric(segment1, final_width = widfuncsegment1, max_points = 8190, tolerance = tol, relative = True, number_of_evaluations = numeval)
    
    #defines a parametric turn with a start angle of spiralendangle + numpy.pi*(turns%2) and an end angle of (numpy.pi + spiralendangle) + spiralendangle + numpy.pi*(turns%2) and constant radius
    #of centerturnradius. The start angle was chosen to keep the turn and previous segment flush due to the archimedian spiral's end face not being vertially aligned.
    def turn1(u):
        theta = -u * (numpy.pi + spiralendangle) + spiralendangle + numpy.pi*(turns%2)
        r = centerturnradius
        x = -r * numpy.sin(theta) + r * numpy.sin(spiralendangle + numpy.pi*(turns%2))
        y = r * numpy.cos(theta)  - r * numpy.cos(spiralendangle + numpy.pi*(turns%2))
        return (x, y)
    
    #defines the first turn's derivitive
    def dturn1_du(u):
        theta = -u * (numpy.pi + spiralendangle) + spiralendangle + numpy.pi*(turns%2)
        r = centerturnradius
        if u > 0:
            dx = -r*(theta/u)*numpy.cos(theta)
            dy = -r*(theta/u)*numpy.sin(theta)
        else:
            dx = (1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi) + radius*radiusratio*numpy.sin((turns+0.5)*numpy.pi)*(turns+0.5)*numpy.pi
            dy = -(1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi) + radius*radiusratio*numpy.cos((turns+0.5)*numpy.pi)*(turns+0.5)*numpy.pi
        return (dx, dy)
    
    #Defines the width of the parametric turn for varous values of u.
    def widfuncturn1(u):
        lengthalongspiral = numpy.pi*(turns+0.5)*0.5*radius*(2-(1 - radiusratio)) + inputoutputhorizontallinelength + inputoutputverticallinelength + inputoutputturnradius*numpy.pi/2 - centerturnradius*dy_dx1 + centerturnradius*(numpy.pi + spiralendangle)*u
        lengthalongunit = (lengthalongspiral + unitremainderlength - centerturnradius*dy_dx1)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #Adds the parametric turn defined above to the end of the TWPAspiralline path. This turn is parametric due to normal path.turn objects only being able to linearly taper their widths instead of 
    #continuously varying them.
    TWPAspirallinewide.parametric(turn1, dturn1_du, tolerance = tol, max_points = 8190, final_width = widfuncturn1, relative = True)
    
    #defines a parametric turn with a start angle of numpy.pi*(turns%2) and an end angle of (numpy.pi + spiralendangle) + numpy.pi*(turns%2) and constant radius
    #of centerturnradius. The end angle was chosen to keep the next segment and next spiral flush due to the archimedian spiral's end face not being vertially aligned.
    def turn2(u):
        r = centerturnradius
        theta = u * (numpy.pi + spiralendangle) + numpy.pi*(turns%2)
        x = -r * numpy.sin(theta) + r*numpy.sin(numpy.pi*(turns%2))
        y = r * numpy.cos(theta) - r*numpy.cos(numpy.pi*(turns%2))
        return (x, y)
    
    #defines the second turn's derivitive
    def dturn2_du(u):
        theta = u * (numpy.pi + spiralendangle)
        r = centerturnradius
        if u < 1:
            dx = -r*(numpy.pi + spiralendangle)*numpy.cos(theta)
            dy = -r*(numpy.pi + spiralendangle)*numpy.sin(theta)
        else:
            dx = radiusratio*radius*(turns+0.5)*numpy.pi
            dy = -(1 - radiusratio)*radius
        return (dx, dy)
    
    #Defines the width of the parametric turn for varous values of u.
    def widfuncturn2(u):
        lengthalongspiral = centerturnradius*(numpy.pi + spiralendangle)*u
        lengthalongunit = (lengthalongspiral + unitremainderlength - centerturnradius*dy_dx1 + centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #Adds a parametric turn defined above to the end of the TWPAspiralline path.
    TWPAspirallinewide.parametric(turn2, dturn2_du, tolerance = tol, max_points = 8190, final_width = widfuncturn2)
    
    #Defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def segment2(u):
        if  (turns%2) == 1:
            r = u*centerturnradius*dy_dx1
        else:
            r = -u*centerturnradius*dy_dx1
        x = r*numpy.cos(spiralendangle)
        y = r*numpy.sin(spiralendangle)

        return (x,y)

    #Defines the width of the parametric segment for varous values of u.
    def widfuncsegment2(u):
        lengthalongspiral = centerturnradius*(numpy.pi + spiralendangle) - centerturnradius*dy_dx1*u
        lengthalongunit = (lengthalongspiral + unitremainderlength - centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #adds the parametric line defined above to the end of the TWPAspiralline path.
    TWPAspirallinewide.parametric(segment2, final_width = widfuncsegment2, tolerance = tol, max_points = 8190, relative = True, number_of_evaluations = numeval)
    
    #defines the second spiral's shape. This spiral starts on its inner radius and spirals outwads to its outer radius.
    def spiral2(u):
        r = radiusratio*radius + (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi + numpy.pi*(turns%2)
        x = r * numpy.sin(theta) - r*numpy.sin(numpy.pi*(turns%2))
        y = -r * numpy.cos(theta) - radiusratio*radius*(turns%2) + radiusratio*radius*((turns+1)%2)
        return (x, y)

    #defines the derivitive of the second spiral. There is a condition that the end of the spiral's normal vector is pointed in the +y direction.
    def dspiral2_du(u):
        r = radiusratio*radius + (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi
        if u == 1:
            dx = 0
            dy = 1
        else:
            if (turns%2) == 0:
                dx = ((1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi*u) + r*numpy.cos((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
                dy = (-(1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi*u) + r*numpy.sin((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
            else:
                dx = -((1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi*u) + r*numpy.cos((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
                dy = -(-(1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi*u) + r*numpy.sin((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
        return (dx, dy)

    #Defines the width as a function of u of the second spiral.
    def widfuncspiral2(u):
        lengthalongspiral = numpy.pi*(turns + 0.5)*0.5*radius*u*(2*radiusratio + (1 - radiusratio)*u)
        lengthalongunit = (lengthalongspiral + unitremainderlength - 2*centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #adds the second spiral defined above to the end of the TWPAspiralline path. 
    TWPAspirallinewide.parametric(spiral2, dspiral2_du, tolerance = tol, max_points = 8190, final_width = widfuncspiral2, relative = True)
    
    #Rotates the TWPAspiralline path pi-spiralcenterangle radians around its center (0, 0) then moves it to the center of the pad.
    TWPAspirallinewide.rotate(numpy.pi)
    TWPAspirallinewide.translate(padlength/2, 0)
    
    #Creates a new path called TWPAoutputline of width widthfunc(0, 0) + 2*gapfunc(0, 0) at (padwidth/2 - radius, 0)
    TWPAoutputlinewide = gdspy.Path(widthfunc(0, 0) + 2*gapfunc(0, 0), (padwidth/2 - radius, 0))

    #Defines a vertical line
    def outputvertialline(u):
        x = 0
        y = -u*(padlength/2 - wirebondcentertoedge - inputoutputturnradius)
        return(x, y)
    
    #defines the width of the output vertical line as a function of u.
    def widfuncoutputverticalline(u):
        lengthalongspiral = singlespirallength - (1 - u)*inputoutputverticallinelength - (inputoutputturnradius*numpy.pi)/2 - inputoutputhorizontallinelength
        lengthalongunit =(lengthalongspiral + unitremainderlength - 2*centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #Adds the vertical line to the path
    TWPAoutputlinewide.parametric(outputvertialline, tolerance = tol, max_points = 8190, final_width= widfuncoutputverticalline, relative = True, number_of_evaluations = numeval)

    #Defines a left turn
    def outputturn(u):
        r = inputoutputturnradius
        theta = u*(numpy.pi/2)
        x = -r*(numpy.cos(theta) - 1)
        y = -r*numpy.sin(theta)
        return(x, y)
    
    #Defines the derivitive of a left turn
    def doutputturn_du(u):
        r = inputoutputturnradius
        theta = u*(numpy.pi/2)
        dx = r*numpy.sin(theta)
        dy = -r*numpy.cos(theta)
        return(dx, dy)

    #Defines the width of a left turn as a function of u.
    def widfuncoutputturn(u):
        lengthalongspiral = singlespirallength - (1 - u)*(inputoutputturnradius*numpy.pi)/2 - inputoutputhorizontallinelength
        lengthalongunit =(lengthalongspiral + unitremainderlength - 2*centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #Adds a left turn to the path.
    TWPAoutputlinewide.parametric(outputturn, doutputturn_du, tolerance = tol, max_points= 8190, final_width= widfuncoutputturn, relative= True)

    #Defines a horizontal ilne
    def outputhorizontalline(u):
        x = u*(inputoutputhorizontallinelength)
        y = 0
        return(x, y)
    
    #Defines the width of a horizontal line as a function of u.
    def widfuncoutputhorizontalline(u):
        lengthalongspiral = singlespirallength - (1 - u)*inputoutputhorizontallinelength
        lengthalongunit = (lengthalongspiral + unitremainderlength + 2*centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio)) + 2*gapfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1) + 2*gapfunc(locationinunit(lengthalongunit), 1)
        return width

    #Adds a horzinontal line to the path.
    TWPAoutputlinewide.parametric(outputhorizontalline, tolerance = tol, max_points = 8190, final_width= widfuncoutputhorizontalline, relative = True, number_of_evaluations = numeval)

    #Adds a segment of length wirebondpadtaperlength that linearly tapers from the end width of the horizontal line to wirebondpadwith + 2*wirebondpadgap*makegroundplane
    TWPAoutputlinewide.segment(wirebondpadtaperlength, final_width = wirebondpadwidth + 2*wirebondpadgap)

    #Adds a segment of length wirebondpadlength and width wirebondpadwidth + 2*wirebondpadgap*makegroundplane
    TWPAoutputlinewide.segment(wirebondpadlength, final_width= wirebondpadwidth + 2*wirebondpadgap)

     #creates the TWPAinputline path object with an initial width of wirebondpadwidth + 2*wirebondpadgap*makegroundplane and a start position of (0, padlength/2 - wirebondcentertoedge) i.e. the top left corner
    TWPAinputlinenarrow = gdspy.Path(wirebondpadwidth, (0, padlength/2 - wirebondcentertoedge))
    
    #adds a straight segment of length wirebondpathlangth in the +x direction to the end of the path. Since a new width was not specified the segment will use the path's initial width
    TWPAinputlinenarrow.segment(wirebondpadlength, '+x')

    #adds a straight segment of length wirebondtaperlength to the end of the TWPAinputline path. 
    #Since a new direction was not specified the segment will be oriented in the same direction as the end of the previous part of the path.
    #Since a final_width was defined, the segment will linearly taper from the final_width of the previous part of the path to the defined final_width of widthfunc(0, 0) + 2*gapfunc(0, 0)
    TWPAinputlinenarrow.segment(wirebondpadtaperlength, final_width = widthfunc(0, 0))

    #widfuncinputoutputhorizontaline is the function which defines horizontal input line's width at a specific value of u. lengthalongspiral is the length of spiral created at a given value of u,
    #and lengthalong unit is how far along a unit cell a given value of u is. If the fraction lengthalongspiral/singlespirallength is less than spiraltaperratio the x value of 
    #(lengthalongspiral/singlespirallength)/spiraltaperratio) is given to widthfunction to taper as defined, if not, the x value of 1 (end of taper) is fed instead. locationinunit(lengthalongunit) gives
    #the n value of the correct segment for a given value of u. This function the returns the correct spiral width for a given value of u.
    def widfuncinputhorizontalline(u):
        lengthalongspiral = u*inputoutputhorizontallinelength
        lengthalongunit = (lengthalongspiral)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1)
        return width

    #This function defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def inputhorizontalline(u):
        x = u*(inputoutputhorizontallinelength)
        y = 0
        return(x, y)

    #Uses the two above functions to create a horizontal line.
    TWPAinputlinenarrow.parametric(inputhorizontalline, tolerance = tol, max_points = 8190, final_width= widfuncinputhorizontalline, relative = True, number_of_evaluations = numeval)

    #widfuncinputoutputturn is the function which defines input turn's width at a specific value of u. lengthalongspiral is the length of spiral created at a given value of u,
    #and lengthalong unit is how far along a unit cell a given value of u is. If the fraction lengthalongspiral/singlespirallength is less than spiraltaperratio the x value of 
    #(lengthalongspiral/singlespirallength)/spiraltaperratio) is given to widthfunction to taper as defined, if not, the x value of 1 (end of taper) is fed instead. locationinunit(lengthalongunit) gives
    #the n value of the correct segment for a given value of u. This function the returns the correct spiral width for a given value of u.
    def widfuncinputturn(u):
        lengthalongspiral = u*(inputoutputturnradius*numpy.pi/2) + inputoutputhorizontallinelength
        lengthalongunit = (lengthalongspiral)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit))
        return width

    #Defines a right turn
    def inputturn(u):
        r = inputoutputturnradius
        theta = u*(numpy.pi/2)
        y = r*(numpy.cos(theta) - 1)
        x = r*numpy.sin(theta)
        return(x, y)
    
    #Defines the derivitive of a right turn
    def dinputturn_du(u):
        r = inputoutputturnradius
        theta = u*(numpy.pi/2)
        dy = -r*numpy.sin(theta)
        dx = r*numpy.cos(theta)
        return(dx, dy)
    
    #Uses the above three functions to create a right turn
    TWPAinputlinenarrow.parametric(inputturn, dinputturn_du, max_points= 8190, final_width= widfuncinputturn, relative= True)

    #widfuncinputoutputverticalline is the function which defines vertical input line's width at a specific value of u. lengthalongspiral is the length of spiral created at a given value of u,
    #and lengthalong unit is how far along a unit cell a given value of u is. If the fraction lengthalongspiral/singlespirallength is less than spiraltaperratio the x value of 
    #(lengthalongspiral/singlespirallength)/spiraltaperratio) is given to widthfunction to taper as defined, if not, the x value of 1 (end of taper) is fed instead. locationinunit(lengthalongunit) gives
    #the n value of the correct segment for a given value of u. This function the returns the correct spiral width for a given value of u.
    def widfuncinputverticalline(u):
        lengthalongspiral = u*inputoutputverticallinelength + inputoutputturnradius*numpy.pi/2 + inputoutputhorizontallinelength
        lengthalongunit = (lengthalongspiral)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit))
        return width

    #This function defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def inputvertialline(u):
        x = 0
        y = -u*(inputoutputverticallinelength)
        return(x, y)

    #Uses the above two functions to create a vertical line
    TWPAinputlinenarrow.parametric(inputvertialline, tolerance = tol, max_points = 8190, final_width= widfuncinputverticalline, relative = True, number_of_evaluations = numeval)
    
    #creates the TWPAspiralline path object with an initial width of widthfunc(0, 0) + 2*gapfunc(0, 0) and a start position of (-radius, 0) which places the center of the spiral at (0, 0)
    TWPAspirallinenarrow = gdspy.Path(widthfunc(0, 0) + 2*gapfunc(0, 0), (-radius, 0))
    
    #spiral2 is the function that defines the spiral's shape. It is a parametric function of u, a variable which varies from 0 to 1. 
    #First the polar equation is defined, then it is translated into cartesian coordinates which are returned by the function
    def spiral1(u):
        r = radius - (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi
        x = -r * numpy.cos(theta) + radius
        y = r * numpy.sin(theta)
        return (x, y)

    #dspiral1_du is the function which defines the spiral's derivitive. The path's Parametric method uses the derivitive to calculated the direction of the vector of the parametric portion of the path.
    #to simplify the start of the spiral a condition has been added which forces dspiral1_du(0) to be (0, 1) so that the start face of the spiral has a normal vector point in the +y direction, 
    #which results in the start face being horizontal, therefore fully connecting to the input portion of the design. If a derivitive function is not defined, the parametric method will calculate the
    #derivitive numerically from the defined function.
    def dspiral1_du(u):
        r = radius - (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi
        if u == 0:
            dx = 0
            dy = 1
        else:
            dx = (1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi*u) + r*numpy.sin((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi
            dy = -(1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi*u) + r*numpy.cos((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi
        return (dx, dy)

    #widfuncspiral1 is the function which defines the spiral's width at a specific value of u. lengthalongspiral is the length of spiral created at a given value of u,
    #and lengthalong unit is how far along a unit cell a given value of u is. If the fraction lengthalongspiral/singlespirallength is less than spiraltaperratio the x value of 
    #(lengthalongspiral/singlespirallength)/spiraltaperratio) is given to widthfunction to taper as defined, if not, the x value of 1 (end of taper) is fed instead. locationinunit(lengthalongunit) gives
    #the n value of the correct segment for a given value of u. This function the returns the correct spiral width for a given value of u.
    def widfuncspiral1(u):
        lengthalongspiral = numpy.pi*(turns+0.5)*0.5*radius*u*(2-(1 - radiusratio)*u) + inputoutputhorizontallinelength + inputoutputverticallinelength + inputoutputturnradius*numpy.pi/2
        lengthalongunit = (lengthalongspiral)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1)
        return width

    #uses the three previously defined functions to add the desired spiral to the end of the TWPAspiralline path. Each of the functions is fed a value of u, which varies from 0 to 1, to sweep out the shape.
    #Since gdsII files only contain polygons and a spiral is a curve which much be approximated by polygons, tolerance is specified as well as the maximum number of vertices a polygon is allowed
    #to have. relaive = True allows the paametric shape to be moved so that it starts at the previous end of the path, but only if u=0 results in each of the defined functions returning (0, 0).
    TWPAspirallinenarrow.parametric(spiral1, dspiral1_du, tolerance = tol, max_points = 8190, final_width = widfuncspiral1, relative = True)
    
    #This function defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def segment1(u):
        if  (turns%2) == 1:
            r = u*centerturnradius*dy_dx1
        else:
            r = -u*centerturnradius*dy_dx1
        x = r*numpy.cos(spiralendangle)
        y = r*numpy.sin(spiralendangle)
        return (x,y)

    #Defines the width of the parametric segment for varous values of u. 
    def widfuncsegment1(u):
        lengthalongspiral = numpy.pi*(turns+0.5)*0.5*radius*(2-(1 - radiusratio)) + inputoutputhorizontallinelength + inputoutputverticallinelength + inputoutputturnradius*numpy.pi/2 - centerturnradius*dy_dx1*u
        lengthalongunit = (lengthalongspiral + unitremainderlength)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1)
        return width

    #adds a parametric line defined above to the end of the TWPAspiralline path. This segment is parametric due to normal path.segment objects only being able to linearly taper their widths instead of 
    #continuously varying them.
    TWPAspirallinenarrow.parametric(segment1, final_width = widfuncsegment1, max_points = 8190, tolerance = tol, relative = True, number_of_evaluations = numeval)
    
    #defines a parametric turn with a start angle of spiralendangle + numpy.pi*(turns%2) and an end angle of (numpy.pi + spiralendangle) + spiralendangle + numpy.pi*(turns%2) and constant radius
    #of centerturnradius. The start angle was chosen to keep the turn and previous segment flush due to the archimedian spiral's end face not being vertially aligned.
    def turn1(u):
        theta = -u * (numpy.pi + spiralendangle) + spiralendangle + numpy.pi*(turns%2)
        r = centerturnradius
        x = -r * numpy.sin(theta) + r * numpy.sin(spiralendangle + numpy.pi*(turns%2))
        y = r * numpy.cos(theta)  - r * numpy.cos(spiralendangle + numpy.pi*(turns%2))
        return (x, y)
    
    #defines the first turn's derivitive
    def dturn1_du(u):
        theta = -u * (numpy.pi + spiralendangle) + spiralendangle + numpy.pi*(turns%2)
        r = centerturnradius
        if u > 0:
            dx = -r*(theta/u)*numpy.cos(theta)
            dy = -r*(theta/u)*numpy.sin(theta)
        else:
            dx = (1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi) + radius*radiusratio*numpy.sin((turns+0.5)*numpy.pi)*(turns+0.5)*numpy.pi
            dy = -(1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi) + radius*radiusratio*numpy.cos((turns+0.5)*numpy.pi)*(turns+0.5)*numpy.pi
        return (dx, dy)
    
    #Defines the width of the parametric turn for varous values of u.
    def widfuncturn1(u):
        lengthalongspiral = numpy.pi*(turns+0.5)*0.5*radius*(2-(1 - radiusratio)) + inputoutputhorizontallinelength + inputoutputverticallinelength + inputoutputturnradius*numpy.pi/2 - centerturnradius*dy_dx1 + centerturnradius*(numpy.pi + spiralendangle)*u
        lengthalongunit = (lengthalongspiral + unitremainderlength - centerturnradius*dy_dx1)%unitlength
        if (lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #Adds the parametric turn defined above to the end of the TWPAspiralline path. This turn is parametric due to normal path.turn objects only being able to linearly taper their widths instead of 
    #continuously varying them.
    TWPAspirallinenarrow.parametric(turn1, dturn1_du, tolerance = tol, max_points = 8190, final_width = widfuncturn1, relative = True)
    
    #defines a parametric turn with a start angle of numpy.pi*(turns%2) and an end angle of (numpy.pi + spiralendangle) + numpy.pi*(turns%2) and constant radius
    #of centerturnradius. The end angle was chosen to keep the next segment and next spiral flush due to the archimedian spiral's end face not being vertially aligned.
    def turn2(u):
        r = centerturnradius
        theta = u * (numpy.pi + spiralendangle) + numpy.pi*(turns%2)
        x = -r * numpy.sin(theta) + r*numpy.sin(numpy.pi*(turns%2))
        y = r * numpy.cos(theta) - r*numpy.cos(numpy.pi*(turns%2))
        return (x, y)
    
    #defines the second turn's derivitive
    def dturn2_du(u):
        theta = u * (numpy.pi + spiralendangle)
        r = centerturnradius
        if u < 1:
            dx = -r*(numpy.pi + spiralendangle)*numpy.cos(theta)
            dy = -r*(numpy.pi + spiralendangle)*numpy.sin(theta)
        else:
            dx = radiusratio*radius*(turns+0.5)*numpy.pi
            dy = -(1 - radiusratio)*radius
        return (dx, dy)
    
    #Defines the width of the parametric turn for varous values of u.
    def widfuncturn2(u):
        lengthalongspiral = centerturnradius*(numpy.pi + spiralendangle)*u
        lengthalongunit = (lengthalongspiral + unitremainderlength - centerturnradius*dy_dx1 + centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #Adds a parametric turn defined above to the end of the TWPAspiralline path.
    TWPAspirallinenarrow.parametric(turn2, dturn2_du, tolerance = tol, max_points = 8190, final_width = widfuncturn2)
    
    #Defines a straight line in u-parametric polar coordinates, which then is transformed into cartesion coordinates which are then returned.
    def segment2(u):
        if  (turns%2) == 1:
            r = u*centerturnradius*dy_dx1
        else:
            r = -u*centerturnradius*dy_dx1
        x = r*numpy.cos(spiralendangle)
        y = r*numpy.sin(spiralendangle)

        return (x,y)

    #Defines the width of the parametric segment for varous values of u.
    def widfuncsegment2(u):
        lengthalongspiral = centerturnradius*(numpy.pi + spiralendangle) - centerturnradius*dy_dx1*u
        lengthalongunit = (lengthalongspiral + unitremainderlength - centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #adds the parametric line defined above to the end of the TWPAspiralline path.
    TWPAspirallinenarrow.parametric(segment2, final_width = widfuncsegment2, tolerance = tol, max_points = 8190, relative = True, number_of_evaluations = numeval)
    
    #defines the second spiral's shape. This spiral starts on its inner radius and spirals outwads to its outer radius.
    def spiral2(u):
        r = radiusratio*radius + (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi + numpy.pi*(turns%2)
        x = r * numpy.sin(theta) - r*numpy.sin(numpy.pi*(turns%2))
        y = -r * numpy.cos(theta) - radiusratio*radius*(turns%2) + radiusratio*radius*((turns+1)%2)
        return (x, y)

    #defines the derivitive of the second spiral. There is a condition that the end of the spiral's normal vector is pointed in the +y direction.
    def dspiral2_du(u):
        r = radiusratio*radius + (1 - radiusratio)*radius*u
        theta = (turns+0.5) * u * numpy.pi
        if u == 1:
            dx = 0
            dy = 1
        else:
            if (turns%2) == 0:
                dx = ((1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi*u) + r*numpy.cos((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
                dy = (-(1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi*u) + r*numpy.sin((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
            else:
                dx = -((1 - radiusratio)*radius*numpy.sin((turns+0.5)*numpy.pi*u) + r*numpy.cos((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
                dy = -(-(1 - radiusratio)*radius*numpy.cos((turns+0.5)*numpy.pi*u) + r*numpy.sin((turns+0.5)*numpy.pi*u)*(turns+0.5)*numpy.pi)
        return (dx, dy)

    #Defines the width as a function of u of the second spiral.
    def widfuncspiral2(u):
        lengthalongspiral = numpy.pi*(turns + 0.5)*0.5*radius*u*(2*radiusratio + (1 - radiusratio)*u)
        lengthalongunit = (lengthalongspiral + unitremainderlength - 2*centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #adds the second spiral defined above to the end of the TWPAspiralline path. 
    TWPAspirallinenarrow.parametric(spiral2, dspiral2_du, tolerance = tol, max_points = 8190, final_width = widfuncspiral2, relative = True)
    
    #Rotates the TWPAspiralline path pi-spiralcenterangle radians around its center (0, 0) then moves it to the center of the pad.
    TWPAspirallinenarrow.rotate(numpy.pi)
    TWPAspirallinenarrow.translate(padlength/2, 0)
    
    #Creates a new path called TWPAoutputline of width widthfunc(0, 0) + 2*gapfunc(0, 0) at (padwidth/2 - radius, 0)
    TWPAoutputlinenarrow = gdspy.Path(widthfunc(0, 0) + 2*gapfunc(0, 0), (padwidth/2 - radius, 0))

    #Defines a vertical line
    def outputvertialline(u):
        x = 0
        y = -u*(padlength/2 - wirebondcentertoedge - inputoutputturnradius)
        return(x, y)
    
    #defines the width of the output vertical line as a function of u.
    def widfuncoutputverticalline(u):
        lengthalongspiral = singlespirallength - (1 - u)*inputoutputverticallinelength - (inputoutputturnradius*numpy.pi)/2 - inputoutputhorizontallinelength
        lengthalongunit =(lengthalongspiral + unitremainderlength - 2*centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1)
        return width

    #Adds the vertical line to the path
    TWPAoutputlinenarrow.parametric(outputvertialline, tolerance = tol, max_points = 8190, final_width= widfuncoutputverticalline, relative = True, number_of_evaluations = numeval)

    #Defines a left turn
    def outputturn(u):
        r = inputoutputturnradius
        theta = u*(numpy.pi/2)
        x = -r*(numpy.cos(theta) - 1)
        y = -r*numpy.sin(theta)
        return(x, y)
    
    #Defines the derivitive of a left turn
    def doutputturn_du(u):
        r = inputoutputturnradius
        theta = u*(numpy.pi/2)
        dx = r*numpy.sin(theta)
        dy = -r*numpy.cos(theta)
        return(dx, dy)

    #Defines the width of a left turn as a function of u.
    def widfuncoutputturn(u):
        lengthalongspiral = singlespirallength - (1 - u)*(inputoutputturnradius*numpy.pi)/2 - inputoutputhorizontallinelength
        lengthalongunit =(lengthalongspiral + unitremainderlength - 2*centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1)
        return width
    
    #Adds a left turn to the path.
    TWPAoutputlinenarrow.parametric(outputturn, doutputturn_du, tolerance = tol, max_points= 8190, final_width= widfuncoutputturn, relative= True)

    #Defines a horizontal ilne
    def outputhorizontalline(u):
        x = u*(inputoutputhorizontallinelength)
        y = 0
        return(x, y)
    
    #Defines the width of a horizontal line as a function of u.
    def widfuncoutputhorizontalline(u):
        lengthalongspiral = singlespirallength - (1 - u)*inputoutputhorizontallinelength
        lengthalongunit = (lengthalongspiral + unitremainderlength + 2*centerturnradius*dy_dx1 + 2*centerturnradius*(numpy.pi + spiralendangle))%unitlength
        if (1 - lengthalongspiral/singlespirallength) < spiraltaperratio:
            width = widthfunc(locationinunit(lengthalongunit), ((1 - lengthalongspiral/singlespirallength)/spiraltaperratio))
        else:
            width = widthfunc(locationinunit(lengthalongunit), 1)
        return width

    #Adds a horzinontal line to the path.
    TWPAoutputlinenarrow.parametric(outputhorizontalline, tolerance = tol, max_points = 8190, final_width= widfuncoutputhorizontalline, relative = True, number_of_evaluations = numeval)

    #Adds a segment of length wirebondpadtaperlength that linearly tapers from the end width of the horizontal line to wirebondpadwith + 2*wirebondpadgap*makegroundplane
    TWPAoutputlinenarrow.segment(wirebondpadtaperlength, final_width = wirebondpadwidth)

    #Adds a segment of length wirebondpadlength and width wirebondpadwidth + 2*wirebondpadgap*makegroundplane
    TWPAoutputlinenarrow.segment(wirebondpadlength, final_width= wirebondpadwidth)

    TWPAinputlinewide = gdspy.boolean(TWPAinputlinewide, TWPAinputlinenarrow, 'not', precision = tol, max_points = 8190)
    TWPAspirallinewide = gdspy.boolean(TWPAspirallinewide, TWPAspirallinenarrow, 'not', precision = tol, max_points = 8190)
    TWPAoutputlinewide = gdspy.boolean(TWPAoutputlinewide, TWPAoutputlinenarrow, 'not', precision = tol, max_points = 8190)

    cell.add(TWPAinputlinewide)
    cell.add(TWPAspirallinewide)
    cell.add(TWPAoutputlinewide)
    
    return

In [6]:
#This Cell defines and executes TWPA making functions. The time it takes to execute this cell is the time it takes to generate the TWPA.

#Creates the center line and the ground plane of a TWPA.
def TWPA():
    centerline(widthfunction, zerofunction, False)
    centerline(widthfunction, gapfunction, True)
    return
TWPA()

def invertedTWPA():
    gapline(widthfunction, gapfunction)
#invertedTWPA()

In [7]:
#This Cell prints some useful information about the generated TWPA
print('The total length of the TWPA is ' + str(round(totallength,0)) + ' um or ' + str(round(totallength/10000,1)) + ' cm.')
print('The length of the thin TWPA line (line between the taper ends) is approximately ' + str(numpy.trunc(thinwirelength)) + ' um or ' + str(round(thinwirelength/10000, 1)) + ' cm.')
print('Each photonic crystal unit has a length of ' + str(unitlength) + ' um.')
print('There are ' + str(totalfullunits) + ' full photonic crystal units in the full TWPA line.')
print('There are ' + str(thinfullunits) + ' full photonic crystal units in the thin TWPA line.')
print('The spacing between the spirals turns is ' + str(round(spacing, 1)) + ' um.')

The total length of the TWPA is 162685.0 um or 16.3 cm.
The length of the thin TWPA line (line between the taper ends) is approximately 2213.0 um or 0.2 cm.
Each photonic crystal unit has a length of 660 um.
There are 246.0 full photonic crystal units in the full TWPA line.
There are 3.0 full photonic crystal units in the thin TWPA line.
The spacing between the spirals turns is 321.4 um.


In [8]:
#This Cell opens the generated TWPA in gdspy's included viewer
gdspy.LayoutViewer(lib)

<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [9]:
#This Cell saves the generated TWPA to a file named whatever is in the quotes
lib.write_gds('Test TWPA')